# GROUP MEMBERS: Ben Klimala, Suvir Grewal, Daniel Zimmer, Liam Dale

## imports

In [1]:
import sys
%pip install gradio pandas matplotlib scikit-learn

import os
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from scipy import stats
from typing import Iterable, Union

Note: you may need to restart the kernel to use updated packages.


/home/ldale/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## utils

In [2]:
def export_to_csv(data: Union[pd.DataFrame, Iterable[dict]], file_path: str) -> pd.DataFrame:
    """Save data to a CSV file.

    Parameters
    ----------
    data : Union[pandas.DataFrame, Iterable[dict]]
        The data to export. Can be a DataFrame or an iterable of dictionaries.
    file_path : str
        Destination path for the CSV file.

    Returns
    -------
    pandas.DataFrame
        DataFrame representation of ``data`` that was saved.
    """
    if isinstance(data, pd.DataFrame):
        df = data
    else:
        df = pd.DataFrame(data)

    df.to_csv(file_path, index=False)
    return df

## recorder_tab

In [3]:
folder_path = "logs"  # Relative folder path
os.makedirs(folder_path, exist_ok=True)  # Ensure folder exists

LOG_FILE = os.path.join(folder_path, "workout_log.csv")  # Relative file path

# ---------- Exercise Dictionary ----------

default_body_part = "Chest"

exercise_dict = {
    "Chest": [
        "Bench Press", "Incline Bench Press", "Dumbbell Press", "Incline Dumbbell Press", "Chest Fly",
        "Cable Crossover", "Push-Up", "Pec Deck", "Dips", "Decline Press"
    ],
    "Back": [
        "Deadlift", "Pull-Up", "Bent-Over Row", "Lat Pulldown", "Seated Row",
        "T-Bar Row", "Cable Row", "Chin-Up", "Straight-Arm Pulldown", "Face Pull"
    ],
    "Quads": ["Squat", "Leg Press", "Lunges", "Bulgarian Split Squat", "Front Squat"],
    "Hamstrings": ["Romanian Deadlift", "Leg Curl", "Good Morning", "Glute Bridge", "Kettlebell Swing"],
    "Shoulders": ["Overhead Press", "Lateral Raise", "Front Raise", "Arnold Press", "Reverse Pec Deck"],
    "Biceps": ["Barbell Curl", "Dumbbell Curl", "Preacher Curl", "Hammer Curl", "Cable Curl"],
    "Triceps": ["Tricep Pushdown", "Skull Crushers", "Overhead Extension", "Close-Grip Bench", "Dips"]
}

default_exercise_dict = {
    "Chest": "Bench Press",
    "Back": "Deadlift",
    "Quads": "Squat",
    "Hamstrings": "Romanian Deadlift",
    "Shoulders": "Overhead Press",
    "Biceps": "Barbell Curl",
    "Triceps": "Tricep Pushdown"
}

# ---------- Data Initialization ----------

# Load existing log if available
if os.path.exists(LOG_FILE):
    log = pd.read_csv(LOG_FILE).to_dict(orient="records")
else:
    log = []

# ---------- Functions ----------

def update_exercises(body_part):
    return gr.update(
        choices=exercise_dict.get(body_part, []),
        value=default_exercise_dict.get(body_part, None)
    )

def add_entry(date, body_part, exercise, sets, reps, weight):
    try:
        datetime.strptime(date, "%Y-%m-%d")
    except ValueError:
        return pd.DataFrame(log)

    entry = {
        "Date": date,
        "Body Part": body_part,
        "Exercise": exercise,
        "Sets": sets,
        "Reps": reps,
        "Weight": weight
    }
    log.append(entry)
    df = export_to_csv(log, LOG_FILE)
    return df

def download_log():
    if not log:
        return None
    export_to_csv(log, LOG_FILE)
    return LOG_FILE

def clear_log():
    global log
    log = []
    return pd.DataFrame()

def reset_log():
    global log
    log = []
    if os.path.exists(LOG_FILE):
        os.remove(LOG_FILE)
    return pd.DataFrame()

# ---------- Gradio UI ----------

today_str = datetime.today().strftime("%Y-%m-%d")

def recorder_app():
    gr.Markdown("## 🏋️ Workout Recorder")
    gr.Markdown("Log your workouts by body part, sets, reps, and weight.")

    date_input = gr.Textbox(label="Date (YYYY-MM-DD)", value=today_str)

    with gr.Row():
        body_part = gr.Dropdown(label="Body Part", choices=list(exercise_dict.keys()), value=default_body_part)
        exercise = gr.Dropdown(label="Exercise", choices=exercise_dict[default_body_part], value=default_exercise_dict[default_body_part])

    body_part.change(fn=update_exercises, inputs=body_part, outputs=exercise)

    with gr.Row():
        sets = gr.Dropdown(label="Sets", choices=list(range(1, 51)), value=3)
        reps = gr.Dropdown(label="Reps", choices=list(range(1, 101)), value=10)
        weight = gr.Dropdown(label="Weight (lbs)", choices=[round(x * 0.5, 1) for x in range(1, 2001)], value=100)

    submit_btn = gr.Button("Log Entry")
    output_table = gr.Dataframe(label="Workout Log")

    submit_btn.click(fn=add_entry, inputs=[date_input, body_part, exercise, sets, reps, weight], outputs=output_table)

    gr.Markdown("### 📁 Download or Manage Your Log")

    # Download section
    download_btn = gr.Button("Download Log as CSV")
    file_output = gr.File(label="Click to Download")

    download_btn.click(fn=download_log, outputs=file_output)

    # Clear and reset buttons
    clear_btn = gr.Button("Clear Log (Clears current entry. Does not affect file)")
    clear_btn.click(fn=clear_log, outputs=output_table)

    reset_btn = gr.Button("Reset Log (Clears current entry + deletes file)")
    reset_btn.click(fn=reset_log, outputs=output_table)

def recorder_tab():
    with gr.TabItem("📝 Recorder"):
        recorder_app()

## routines_tab

In [4]:
# Define exercises for the workout routine builder
exercise_dict = {
    "Warm-Up": [
        "Arm Circles",
        "Hip Circles",
        "Leg Swings (Front-to-Back and Side-to-Side)",
        "March in Place",
        "Jog in Place",
        "Shoulder Rolls",
        "Bodyweight Squats",
        "Runner’s Lunge with Twist",
        "Squat to Reach",
        "Side Reaches",
        "Toe Touches",
        "Bird Dogs",
        "Glute Bridges",
        "Inchworms",
        "Carioca",
        "Squat Matrix",
    ],
    "Yoga/Stretching": [
        "Child’s Pose",
        "Cat-Cow",
        "Thread the Needle",
        "Downward Facing Dog",
        "Cobra Pose",
        "Chair Pose",
        "Upward Facing Dog",
        "Low Lunge Twist",
        "Low Warrior with Hands Behind Back",
        "Half Split",
        "Seated Forward Fold",
        "Reclining Hand-to-Big-Toe Pose",
        "Toes Squat to Ankle Stretch",
        "Standing Forward Fold",
        "Triangle Pose",
        "Butterfly Stretch",
    ],
    "Strength": [
        "Barbell Back Squat",
        "Dumbbell Chest Press",
        "Barbell Prone Row",
        "Kettlebell Romanian Deadlift",
        "Push-Up",
        "Pull-Up",
        "Dumbbell Row",
        "Single-Leg Deadlift",
        "Hip Lift",
        "Glute Bridge",
        "Bulgarian Split Squat",
        "Leg Press",
        "Leg Extension",
        "Wall Sit",
        "Deadlift",
        "Stiff-Legged Deadlift",
        "Leg Curl",
        "Standing Calf Raise",
        "Seated Calf Raise",
        "Bench Press",
        "Incline Bench Press",
        "Decline Bench Press",
        "Chest Fly",
        "Cable Crossover",
        "Dips",
        "Lat Pulldown",
        "Chin-Up",
        "Bent-Over Row",
        "Cable Row",
        "Upright Row",
        "Shoulder Press",
        "Military Press",
        "Arnold Press",
        "Lateral Raise",
        "Front Raise",
        "Triceps Pushdown",
        "Triceps Extension",
        "Preacher Curl",
        "Barbell Curl",
        "Hammer Curl",
        "Zottman Curl",
        "Crunch",
        "Reverse Crunch",
        "Russian Twist",
        "Leg Raise",
        "Plank",
        "Back Extension",
    ],
    "Cardio": [
        "Jump Rope",
        "Dancing",
        "Power Walking",
        "Swimming",
        "Boxing",
        "Jogging in Place",
        "Air Jump Rope",
        "Jumping Jacks",
        "Squat to Front Kick",
        "Stair Climb",
        "Lateral Shuffles",
        "Mountain Climbers",
        "Burpees",
        "High Knees",
        "Sled Push",
        "Cycling Sprints",
        "Rowing Machine",
        "Explosive Circuits",
        "HIIT",
        "Step Mill",
    ],
}

# Lists to hold staged and published workout routines
full_plan = pd.DataFrame(columns=["Section", "Category", "Exercise", "Sets", "Reps"])
staged_plan = pd.DataFrame(columns=["Section", "Category", "Exercise", "Sets", "Reps"])

# File path for exporting published routines
folder_path = "logs"
os.makedirs(folder_path, exist_ok=True)
ROUTINE_FILE = os.path.join(folder_path, "workout_routines.csv")

def get_exercises(category: str):
    """Return available exercises for the selected category."""
    return gr.update(choices=exercise_dict[category], value=exercise_dict[category][0])

def add_workout(section_label: str, category: str, exercise: str, sets: str, reps: str):
    """Add a workout entry to the staged plan."""
    global staged_plan
    row = {
        "Section": section_label,
        "Category": category,
        "Exercise": exercise,
        "Sets": sets,
        "Reps": reps,
    }
    staged_plan.loc[len(staged_plan)] = row
    return format_staged_plan()

def format_staged_plan():
    """Return the staged plan DataFrame with an index column for easy deletion."""
    return staged_plan.reset_index().rename(columns={"index": "Index"})

def delete_entry(index: str):
    """Delete an entry from the staged plan by index."""
    global staged_plan
    try:
        staged_plan.drop(index=int(index), inplace=True)
        staged_plan.reset_index(drop=True, inplace=True)
    except Exception:
        pass
    return format_staged_plan()

def clear_plan():
    """Clear the staged plan."""
    global staged_plan
    staged_plan = staged_plan.iloc[0:0].copy()
    return format_staged_plan()

def publish_plan():
    """Publish the staged plan to the final plan and save to CSV."""
    global full_plan
    full_plan = staged_plan.copy()
    export_to_csv(full_plan, ROUTINE_FILE)
    return full_plan


def download_plan():
    """Return the CSV file containing the published plan for download."""
    if full_plan.empty:
        return None
    export_to_csv(full_plan, ROUTINE_FILE)
    return ROUTINE_FILE

def reset_plan():
    """Reset both staged and published plans."""
    global staged_plan, full_plan
    staged_plan = staged_plan.iloc[0:0].copy()
    full_plan = full_plan.iloc[0:0].copy()
    if os.path.exists(ROUTINE_FILE):
        os.remove(ROUTINE_FILE)
    return format_staged_plan(), full_plan

def routine_builder():
    """Create the workout routine builder interface."""

    gr.Markdown("## ⌛ Workout Routine Builder")
    gr.Markdown("### Create your custom workout routine by adding exercises to the staged plan. You can then publish it to finalize your workout plan.")

    with gr.Row():
        section_label = gr.Dropdown(
            label="Section Label",
            choices=["Warmup", "Explosive Exercises", "Strength Set", "Cardio"],
            value="Warmup",
        )
        category_dropdown = gr.Dropdown(
            label="Category", choices=list(exercise_dict.keys()), value="Warm-Up"
        )
        exercise_dropdown = gr.Dropdown(
            label="Exercise", choices=exercise_dict["Warm-Up"]
        )

    with gr.Row():
        sets_dropdown = gr.Dropdown(
            label="Sets", choices=["None"] + [str(i) for i in range(1, 6)], value="None"
        )
        reps_dropdown = gr.Dropdown(
            label="Reps",
            choices=["None"] + [str(i * 2) for i in range(1, 9)],
            value="None",
        )

    add_button = gr.Button("➕ Add to Staged Plan")
    clear_button = gr.Button("🗑️ Clear Staged Plan")
    publish_button = gr.Button("📣 Publish Workout Plan")
    reset_button = gr.Button("🔄 Reset Plans")

    staged_output = gr.DataFrame(
        headers=["Index", "Section", "Category", "Exercise", "Sets", "Reps"],
        label="Staged Workout Plan",
        interactive=True,
    )
    delete_index = gr.Textbox(
        label="Index to Delete", placeholder="Enter index number to delete"
    )
    delete_button = gr.Button("❌ Delete Entry")
    published_output = gr.DataFrame(
        headers=["Section", "Category", "Exercise", "Sets", "Reps"],
        label="Final Published Plan",
        interactive=False,
    )

    gr.Markdown("### 📁 Download or Manage Your Plan")
    download_button = gr.Button("Download Plan as CSV")
    file_output = gr.File(label="Click to Download")

    category_dropdown.change(
        fn=get_exercises, inputs=category_dropdown, outputs=exercise_dropdown
    )
    add_button.click(
        fn=add_workout,
        inputs=[section_label, category_dropdown, exercise_dropdown, sets_dropdown, reps_dropdown],
        outputs=staged_output,
    )
    delete_button.click(fn=delete_entry, inputs=delete_index, outputs=staged_output)
    clear_button.click(fn=clear_plan, outputs=staged_output)
    publish_button.click(fn=publish_plan, outputs=published_output)
    download_button.click(fn=download_plan, outputs=file_output)
    reset_button.click(fn=reset_plan, outputs=[staged_output, published_output])

def routines_tab():
    """Tab wrapper used in the main application."""
    with gr.TabItem("⌛ Routines"):
        routine_builder()


## prediction_tab

In [5]:
def track_exercise_weights(file_path, exercise_name):
    try:
        # Load the CSV file
        df = pd.read_csv(file_path.name)
        
        # Filter data for the specified exercise
        filtered_df = df[df['Exercise'] == exercise_name]
        
        if filtered_df.empty:
            return f"No data found for exercise: {exercise_name}"
        
        # Convert date strings to datetime objects
        filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
        
        # Sort by date
        filtered_df = filtered_df.sort_values('Date')
        
        # Extract data for plotting
        date = filtered_df['Date']
        pounds = filtered_df['Weight']
        
        # Convert dates to numeric values for regression
        date_nums = mdates.date2num(date)
        
        # Calculate linear regression
        slope, intercept, r_value, p_value, std_err = stats.linregress(date_nums, pounds)
        
        # Create regression line
        regression_line = slope * date_nums + intercept
        
        # Create the plot
        plt.figure(figsize=(12, 6))
        plt.title(f'{exercise_name} Weight Progress')
        plt.xlabel('Date')
        plt.ylabel('Weight (lbs)')
        plt.grid(True)
        plt.xticks(rotation=45)
        
        # Plot scatter points only (no line connecting them)
        plt.scatter(date, pounds, color='blue', label='Data points')
        
        # Add regression line
        plt.plot(date, regression_line, color='red', label=f'Trend') 
        
        plt.legend()
        plt.tight_layout()
        
        # Calculate statistics
        stats_data = {
            "Current Weight": f"{filtered_df['Weight'].iloc[-1]} lbs",
            "Starting Weight": f"{filtered_df['Weight'].iloc[0]} lbs",
            "Progress": f"{filtered_df['Weight'].iloc[-1] - filtered_df['Weight'].iloc[0]} lbs",
            "Average Weight": f"{filtered_df['Weight'].mean():.2f} lbs",
            "Max Weight": f"{filtered_df['Weight'].max()} lbs"
        }
        
        stats_text = "\n".join([f"{k}: {v}" for k, v in stats_data.items()])
        
        return plt.gcf(), stats_text
    
    except Exception as e:
        return None, f"Error: {str(e)}"

def prediction_app():
    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload CSV File")
            exercise_input = gr.Textbox(label="Exercise Name")
            submit_btn = gr.Button("Track Progress")
        
        with gr.Column():
            stats_output = gr.Textbox(label="Statistics", lines=6)
            plot_output = gr.Plot(label="Progress Chart")
    
    submit_btn.click(
        fn=track_exercise_weights,
        inputs=[file_input, exercise_input],
        outputs=[plot_output, stats_output]
    )
    
    gr.Markdown("""
    ## CSV Format Requirements
    Your CSV file should have:
    - A 'Date' column in a format like 'YYYY-MM-DD'
    - An 'Exercise' column with the name of each exercise
    - A 'Weight' column with the weight values in lbs
    
    Example:
    """)

def prediction_tab():
    with gr.TabItem("🔮 Prediction"):
            gr.Markdown("# Exercise Weight Predictor")
            gr.Markdown("Upload a CSV file with exercise data and select an exercise to track progress and predict where you'll be.")
            prediction_app()

## app

In [6]:
with gr.Blocks() as demo:
    gr.Markdown("# 🏋️‍♂️ RepShare - Workout Toolkit")
    with gr.Tabs():
        recorder_tab()
        routines_tab()
        prediction_tab()


demo.launch()

/home/ldale/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Chest or set allow_custom_value=True.
  warnings.warn(


KeyError: 'Chest'